In [1]:
import json
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder
import tensorflow as tf
from sklearn.preprocessing import StandardScaler
from google.colab import drive
import os
import librosa
from librosa.feature import spectral_bandwidth

# Mount Google Drive for access
drive.mount('/content/drive')

def extract_5sec_chunks(audio_array: np.ndarray, window_size_s: float = 8.0, hop_size_s: float = 4.0, sample_rate=16000) -> np.ndarray:
    frame_length = int(window_size_s * sample_rate)
    frame_step = int(hop_size_s * sample_rate)
    framed_audio = tf.signal.frame(audio_array, frame_length, frame_step, pad_end=False)
    return framed_audio

def spectral_bandwidth_feature(audio_array, sample_rate=22050, n_fft=2048, p=2):
    spec_bandwidth = spectral_bandwidth(y=audio_array, sr=sample_rate, n_fft=n_fft, p=p)
    return spec_bandwidth.T  # Transpose the array to have shape (n_samples, n_features)

def spectral_bandwidth_chunks(dataframe):
    y = []
    X = []

    for i, row in dataframe.iterrows():
        label = row['primary_label']
        tensor_5sec = row['audio_chunks']

        for each in tensor_5sec:
            y.append(label)
            each = np.array(each)
            X_spec_bandwidth = spectral_bandwidth_feature(each)  # Compute Spectral Bandwidth
            X_spec_bandwidth = X_spec_bandwidth.reshape(1, -1)  # Reshape to have a single sample dimension
            X.append(X_spec_bandwidth)

    assert len(y) == len(X)

    y = np.array(y)
    X = np.concatenate(X, axis=0)

    return X, y

dataset_path = '/content/drive/MyDrive/207-Project/notebooks/RG/3_species/'
metadata_path = os.path.join(dataset_path, "train_val.csv")
metadata_df = pd.read_csv(metadata_path)

train_df = metadata_df[metadata_df['data'] == 'train'].copy()
validate_df = metadata_df[metadata_df['data'] == 'val'].copy()

train_audios = []
for filename in train_df['filename_npy']:
    audio = np.load('/content/drive/MyDrive/207-Project/data/train/librosa_loaded/' + filename)
    train_audios.append(audio)

val_audios = []
for filename in validate_df['filename_npy']:
    audio = np.load('/content/drive/MyDrive/207-Project/data/train/librosa_loaded/' + filename)
    val_audios.append(audio)

train_chunks = [extract_5sec_chunks(audio) for audio in train_audios]
val_chunks = [extract_5sec_chunks(audio) for audio in val_audios]

train_df['audio_chunks'] = train_chunks
validate_df['audio_chunks'] = val_chunks

X_train, y_train = spectral_bandwidth_chunks(train_df)
X_val, y_val = spectral_bandwidth_chunks(validate_df)

X_train_list = X_train.tolist()
y_train_list = y_train.tolist()
X_val_list = X_val.tolist()
y_val_list = y_val.tolist()

with open('/content/drive/MyDrive/My-207/X_train_bandwidth.json', 'w') as file:
    json.dump(X_train_list, file)

with open('/content/drive/MyDrive/My-207/y_train_bandwidth.json', 'w') as file:
    json.dump(y_train_list, file)

with open('/content/drive/MyDrive/My-207/X_val_bandwidth.json', 'w') as file:
    json.dump(X_val_list, file)

with open('/content/drive/MyDrive/My-207/y_val_bandwidth.json', 'w') as file:
    json.dump(y_val_list, file)


Mounted at /content/drive
